# Introduction to NLP: Assignment 2 

## Assignment on Text Classification and Sequence Labeling

### Description of Assignment 2

This assignment relates to the Text classification and Sequence labeling themes of the introduction to NLP (courses Deskriptiv analytik / Machine learning for descriptive problems), and will focus on gaining some practical, hands-on experience in building and training simple models for these tasks.

The assignment is handed in as a Jupyternotebook (or a PDF render thereof) containing the code used to solve the problem, output presenting the results, and, most importantly, notes that present the students' conclusions and answer questions posed in the assignment.

**Assignment steps/Questions:**

1. Test sklearn’s TfidfVectorizer in place of CountVectorizer on the IMDB data. Do you see any difference in the classification results or the optimal C value?

2. Test different lengths of n-grams in the CountVectorizer on the IMDB data. Do you see any difference in the classification results or the optimal C value ? Do these n-grams show up also in the list of most significant positive/negative features?

3. In the data package for the course [http://dl.turkunlp.org/intro-to-nlp.tar.gz](http://dl.turkunlp.org/intro-to-nlp.tar.gz), the directory language_identification contains data for 5 languages. Based on this data, train an SVM classifier for language recognition between these 5 languages.

4. If you completed (3), toy around with features, especially the ngram_range and analyzer parameters, which allow you to test classification based on character ngrams of various lengths (not only word n-grams). Gain some insight in to the accuracy of the classifier with different features, and try to identify misclassified documents -why do you think they were misclassified?

5. **BONUS** On the address universaldependencies.org, you will find datasets for a bunch of languages. These come in an easy-to-parse, well-documented format. Pick one language that interests you, and one treebank for that language, and try to builda POS tagger for this language. You can use the 4th column “UPOS” [https://universaldependencies.org/format.html](https://universaldependencies.org/format.html) Report on your findings. If you have extra time, try to experiment with various features and see if you can make your accuracy go up. You can check here [https://universaldependencies.org/conll18/results-upos.html](https://universaldependencies.org/conll18/results-upos.html) what the state of the art roughly is for your selected language and treebank. Did you come close?




### Import libraries

In [1]:
import json
import random
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import sklearn.svm
import sklearn.metrics
import numpy

### Read in IMDB data

In [2]:
with open("imdb_train.json") as f:
    data = json.load(f)
    
random.seed(10) # Seed to replicate same scenario for development
random.shuffle(data) # Shuffle data 

# Preview of data
print("class label:", data[0]["class"])
print("text:", data[0]["text"])

class label: neg
text: the single worst film i've ever seen in a theater. i saw this film at the austin film festival in 2004, and it blew my mind that this film was accepted to a festival. it was an interesting premise, and seemed like it could go somewhere, but just fell apart every time it tried to do anything. first of all, if you're going to do a musical, find someone with musical talent. the music consisted of cheesy piano playing that sounded like they were playing it on a stereo in the room they were filming. the lyrics were terribly written, and when they weren't obvious rhymes, they were groan-inducing rhymes that showed how far they were stretching to try to make this movie work. and you'd think you'd find people who could sing when making a musical, right? not in this case. luckily they were half talking/half singing in rhyme most of the time, but when they did sing it made me cringe. especially when they attempted to sing in harmony. and that just addresses the music. some

### Separate texts and labels 

In [3]:
# We need to gather the texts and labels into separate lists
texts=[d["text"] for d in data]
labels=[d["class"] for d in data]
print("Amount of texts:", len(texts))
print("Amount of labels", len(labels))
print()
for label, text in list(zip(labels, texts))[:10]:
    print(label, text[:50] + "...")

Amount of texts: 25000
Amount of labels 25000

neg the single worst film i've ever seen in a theater....
pos I think the reason for all the opinionated diarrhe...
neg This movie is horrible! It rivals \Ishtar\" in the...
neg This may not be the worst comedy of all time, but ...
pos I found this film to funny from the start. John Wa...
pos The problem is that the movie rode in on the coatt...
neg I was so looking forward to seeing this when it wa...
neg I actually saw this movie in the theater back in i...
neg blows my mind how this movie got made. i watched i...
neg Amateurish in the extreme. Camera work especially ...


## 1. Test sklearn’s TfidfVectorizer

**Test sklearn’s TfidfVectorizer in place of CountVectorizer on the IMDB data. Do you see any difference in the classification results or the optimal C value?**

### Datasplit

In [4]:
train_texts, dev_texts, train_labels, dev_labels = train_test_split(texts, labels, test_size = 0.2)

### Create method for easier showing results in changes of n-grams and C-value

In [5]:
def createClassifier(ngramRange, maxFeatures, cValue, vectorizerType):
    # Change vectorizer type based on variable
    if(vectorizerType == "Count"):
        vectorizer = CountVectorizer(max_features = maxFeatures, binary=True, ngram_range = ngramRange)
    if(vectorizerType == "Idf"):
        vectorizer = TfidfVectorizer(max_features = maxFeatures, binary=True, ngram_range = ngramRange)
    
    feature_matrix_train = vectorizer.fit_transform(train_texts)
    feature_matrix_dev = vectorizer.transform(dev_texts)
    
    classifier = sklearn.svm.LinearSVC(C =  cValue, verbose = 0)
    classifier.fit(feature_matrix_train, train_labels)
    
    print("Vectorizer type={0}, C={1}, n-gram={2}".format(vectorizerType, cValue, ngramRange))
    print("DEV", classifier.score(feature_matrix_dev, dev_labels))
    print("TRAIN", classifier.score(feature_matrix_train, train_labels))
    print()

### CountVectorizer

In [6]:
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.0005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.05, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.5, 
    vectorizerType = "Count"
)

Vectorizer type=Count, C=0.0005, n-gram=(1, 1)
DEV 0.873
TRAIN 0.89445

Vectorizer type=Count, C=0.005, n-gram=(1, 1)
DEV 0.8902
TRAIN 0.95705

Vectorizer type=Count, C=0.05, n-gram=(1, 1)
DEV 0.8784
TRAIN 0.99605

Vectorizer type=Count, C=0.5, n-gram=(1, 1)
DEV 0.864
TRAIN 1.0



### TfidfVectorizer

In [7]:
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.0005, 
    vectorizerType = "Idf"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.005, 
    vectorizerType = "Idf"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.05, 
    vectorizerType = "Idf"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.5, 
    vectorizerType = "Idf"
)

Vectorizer type=Idf, C=0.0005, n-gram=(1, 1)
DEV 0.8396
TRAIN 0.84415

Vectorizer type=Idf, C=0.005, n-gram=(1, 1)
DEV 0.8568
TRAIN 0.8668

Vectorizer type=Idf, C=0.05, n-gram=(1, 1)
DEV 0.8892
TRAIN 0.92155

Vectorizer type=Idf, C=0.5, n-gram=(1, 1)
DEV 0.8972
TRAIN 0.9838



#### Comparing TfidfVectorizer and CountVectorizer when changing C value 

First thing I noticed was that the classification results where very similar. The Count Vectorizers dev and train results are higher on a lower C value like 0.0005. When the C value is increased the Count Vectorizers Train rises to 0.95 very fast and even to 1.0 meanwhile the dev results go up to 0.88 and down to 0.86 when C value is 0.5. The optimal C value for the count vectorizer is probably around 0.005 where the data has not been overfitted to the train data. 

As mentioned above the Count vectorizers results where higher on a lower C value. The Tfidf Vectorizer with a C value of 0.0005 has results around 0.84 on dev and train. When the C Value is increased, both the dev and train results increase steadily. Only after a C value as high as 0.5 is where the dev and train start to separate eachother. I'd say that the optimal C value for the Tfidf Vectorizer is around 0.05 because of diminishing returns on the dev result. This method seems to do a better job with not overfitting with the data. 

The Tfidf Vectorizer when it is run with an optimal C value has better dev results than Count Vectorizer.

## 2. Test different lengths of n-grams in the CountVectorizer on the IMDB data

**Test different lengths of n-grams in the CountVectorizer on the IMDB data. Do you see any difference in the classification results or the optimal C value ? Do these n-grams show up also in the list of most significant positive/negative features?**


Lets start by looking att the results of different n-grams and try to find their optimal C values

### CountVectorizer n-gram (1-1)

In [8]:
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.0005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.05, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000, 
    cValue = 0.5, 
    vectorizerType = "Count"
)

Vectorizer type=Count, C=0.0005, n-gram=(1, 1)
DEV 0.873
TRAIN 0.89445

Vectorizer type=Count, C=0.005, n-gram=(1, 1)
DEV 0.8902
TRAIN 0.95705

Vectorizer type=Count, C=0.05, n-gram=(1, 1)
DEV 0.8784
TRAIN 0.99605

Vectorizer type=Count, C=0.5, n-gram=(1, 1)
DEV 0.864
TRAIN 1.0



### CountVectorizer n-gram (1-2)

In [9]:
createClassifier(
    ngramRange = (1,2), 
    maxFeatures = 100000, 
    cValue = 0.0005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,2), 
    maxFeatures = 100000, 
    cValue = 0.005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,2), 
    maxFeatures = 100000, 
    cValue = 0.05, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,2), 
    maxFeatures = 100000, 
    cValue = 0.5, 
    vectorizerType = "Count"
)

Vectorizer type=Count, C=0.0005, n-gram=(1, 2)
DEV 0.8878
TRAIN 0.93365

Vectorizer type=Count, C=0.005, n-gram=(1, 2)
DEV 0.9016
TRAIN 0.9941

Vectorizer type=Count, C=0.05, n-gram=(1, 2)
DEV 0.896
TRAIN 1.0



/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Vectorizer type=Count, C=0.5, n-gram=(1, 2)
DEV 0.894
TRAIN 1.0



### CountVectorizer n-gram (1-3)

In [10]:
createClassifier(
    ngramRange = (1,3), 
    maxFeatures = 100000, 
    cValue = 0.0005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,3), 
    maxFeatures = 100000, 
    cValue = 0.005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,3), 
    maxFeatures = 100000, 
    cValue = 0.05, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (1,3), 
    maxFeatures = 100000, 
    cValue = 0.5, 
    vectorizerType = "Count"
)

Vectorizer type=Count, C=0.0005, n-gram=(1, 3)
DEV 0.8906
TRAIN 0.9398

Vectorizer type=Count, C=0.005, n-gram=(1, 3)
DEV 0.9026
TRAIN 0.99595

Vectorizer type=Count, C=0.05, n-gram=(1, 3)
DEV 0.8974
TRAIN 1.0



/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Vectorizer type=Count, C=0.5, n-gram=(1, 3)
DEV 0.8934
TRAIN 1.0



### CountVectorizer n-gram (2-3)

In [11]:
createClassifier(
    ngramRange = (2,3), 
    maxFeatures = 100000, 
    cValue = 0.0005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (2,3), 
    maxFeatures = 100000, 
    cValue = 0.005, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (2,3), 
    maxFeatures = 100000, 
    cValue = 0.05, 
    vectorizerType = "Count"
)
createClassifier(
    ngramRange = (2,3), 
    maxFeatures = 100000, 
    cValue = 0.5, 
    vectorizerType = "Count"
)

Vectorizer type=Count, C=0.0005, n-gram=(2, 3)
DEV 0.8606
TRAIN 0.92105

Vectorizer type=Count, C=0.005, n-gram=(2, 3)
DEV 0.8816
TRAIN 0.99445

Vectorizer type=Count, C=0.05, n-gram=(2, 3)
DEV 0.8678
TRAIN 1.0

Vectorizer type=Count, C=0.5, n-gram=(2, 3)
DEV 0.8618
TRAIN 1.0



### Results of different n-grams

When the n-grams are increased the C Value does not need to be that big. The Classifiers results are almost optimal at a value of 0.0005. The n-grams become very fast overfitted to the  train data if the C value is too high. I'd say the most optimal was an n-gram range of (1,2) and the C value as 0.005. 

## Do these n-grams show up also in the list of most significant positive/negative features?

Lets create a mehtod for showing the most significant features. After that we can look at the n-gram ranges of (1,2) and (2,3) since (1,1) will only consist of 1 lenght features.

### Create method for showing significant Features based on the classifier

In [12]:
def showSignificantFeatures(maxFeatures, ngramRange, cValue):
    
    vectorizer = CountVectorizer(max_features = maxFeatures, binary=True, ngram_range = ngramRange)
    feature_matrix_train = vectorizer.fit_transform(train_texts)
    feature_matrix_dev = vectorizer.transform(dev_texts)
    
    classifier = sklearn.svm.LinearSVC(C =  cValue, verbose = 0)
    classifier.fit(feature_matrix_train, train_labels)
    
    index2feature={}
    for feature,idx in vectorizer.vocabulary_.items():
        assert idx not in index2feature #This really should hold
        index2feature[idx]=feature
    
    indices=numpy.argsort(classifier.coef_[0])
    print(indices)
    for idx in indices[:30]:
        print(index2feature[idx])
    print("-------------------------------")
    for idx in indices[::-1][:30]:
        print(index2feature[idx])


### N-gram range (1,2)

In [13]:
print("N-gram (1,2)")
showSignificantFeatures(
    ngramRange = (1,2), 
    maxFeatures = 100000,
    cValue = 0.005
)

N-gram (1,2)
[ 9727 98461 13456 ... 34272 63854 27397]
awful
worst
boring
waste
disappointing
disappointment
dull
poorly
the worst
bad
poor
horrible
terrible
unfortunately
not worth
badly
worse
lame
annoying
lacks
at best
save
stupid
laughable
waste of
oh
avoid
mess
not even
basically
-------------------------------
excellent
perfect
great
wonderful
enjoyable
superb
amazing
incredible
better than
enjoyed
today
pretty good
fun
must see
brilliant
10 10
perfectly
simple
realistic
well worth
definitely worth
beautiful
definitely
refreshing
highly
gem
rare
job
wonderfully
fantastic


### N-gram range (2,3)

In [14]:
print("N-gram (2,3)")
showSignificantFeatures(
    ngramRange = (2,3), 
    maxFeatures = 100000,
    cValue = 0.005
)

N-gram (2,3)
[81494 93036  9743 ... 94089 52062 39579]
the worst
waste of
at best
not even
of the worst
not worth
than this
unless you
bad movie
at all
bad acting
is awful
so bad
worse than
is bad
worst movie
boring and
fails to
is terrible
might have
attempt to
bad and
even worse
looks like
way too
not good
not very
the original
is nothing
much better
-------------------------------
is great
must see
well worth
highly recommended
the best
an excellent
10 10
is excellent
my favorite
enjoyed it
was great
loved it
definitely worth
highly recommend
loved this
great movie
very good
love this
great job
enjoyed this
very well
great film
on dvd
is perfect
is beautiful
fun and
watch it
was excellent
of the best
love it


### The most significant n-grams

It seems to be that, the longer the featue, the more unique and more rare it is. If we look at the n-gram range of (1,2), then the most significant features have some words that are 2 words long. The majority of the features are still of one length. 

If we look at the n-gram range (2,3), then this proves also our point. There are some features that consist of 3 words, but most of the features are 2 words. The most common 3 words consists of the words "of the worst/best". 

It is rare that longer sentences would occur several times in normal text or literature. 

## 3. Train an SVM classifier for language recognition

**In the data package for the course [http://dl.turkunlp.org/intro-to-nlp.tar.gz](http://dl.turkunlp.org/intro-to-nlp.tar.gz), the directory language_identification contains data for 5 languages. Based on this data, train an SVM classifier for language recognition between these 5 languages.**

### Create methods for reading in the data

In [15]:
def openTextFile(lang, section):
    path = "./language-identification/{0}_{1}.txt".format(lang, section)
    textList = []
    
    with open(path) as f:
        for line in f:
            obj = {'lang': lang,
               'text': line.strip()
            }
            textList.append(obj)
            
    return textList
            

In [16]:
def separateLabelText(data):
    texts=[d["text"] for d in data]
    labels=[d["lang"] for d in data]
    
    return texts, labels

In [17]:
def loadLangFiles(languages):
    trainData = []
    develData = []
    testData = []
    
    for lang in languages:
        trainData.extend(openTextFile(lang, 'train'))
        develData.extend(openTextFile(lang, 'devel'))
        testData.extend(openTextFile(lang, 'test'))

    # Shuffle Data
    random.seed(10) # Seed to replicate same scenario for development
    random.shuffle(trainData) # Shuffle data 
    random.shuffle(develData) # Shuffle data 
    random.shuffle(testData) # Shuffle data 
    
    # Separate labels from text
    trainTexts, trainLabels = separateLabelText(trainData)
    develTexts, develLabels = separateLabelText(develData)
    testTexts, testLabels = separateLabelText(testData)

        
    return trainTexts, trainLabels, develTexts, develLabels, testTexts, testLabels 

### Create feature matrix and classifier

In [18]:
languages = ['en', 'es', 'et', 'fi', 'pt']
# Read in the texts and labels
lang_text_train, lang_label_train, lang_text_dev, lang_label_dev, lang_text_test, lang_label_test = loadLangFiles(languages)

In [19]:
lang_vectorizer = TfidfVectorizer(max_features = 100000, binary=True, ngram_range = (1,1))
lang_feature_matrix_train = lang_vectorizer.fit_transform(lang_text_train)
lang_feature_matrix_dev = lang_vectorizer.transform(lang_text_dev)
lang_feature_matrix_test = lang_vectorizer.transform(lang_text_test)

lang_classifier = sklearn.svm.LinearSVC(C = 0.05, verbose = 0)
lang_classifier.fit(lang_feature_matrix_train, lang_label_train)

print("DEV", lang_classifier.score(lang_feature_matrix_dev, lang_label_dev))
print("TEST", lang_classifier.score(lang_feature_matrix_test, lang_label_test))
print("TRAIN", lang_classifier.score(lang_feature_matrix_train, lang_label_train))

DEV 0.934
TEST 0.9392
TRAIN 0.9878


### Try predicting languages

In [20]:
def predictLang(text, vectorizer, classifier):
    data = vectorizer.transform([text])
    
    prediction = classifier.predict(data)
    
    print(prediction)

In [21]:
predictLang("Today is a nice day.", lang_vectorizer, lang_classifier)
predictLang("Estas obras son realizadas con libros, álbumes de música o periódicos como soporte.", lang_vectorizer, lang_classifier)
predictLang("Publik teeb nii mitu korda ja on juba üles köetud.", lang_vectorizer, lang_classifier)
predictLang("Tänään on mahti päivä mennä kävelylle.", lang_vectorizer, lang_classifier)
predictLang("Conseguir um bom exclusivo pode significar a entrada de milhões de dólares em publicidade.", lang_vectorizer, lang_classifier)

['en']
['es']
['et']
['fi']
['pt']


## 4. Toy around with features, especially the ngram_range and analyzer parameters

**If you completed (3), toy around with features, especially the ngram_range and analyzer parameters, which allow you to test classification based on character ngrams of various lengths (not only word n-grams). Gain some insight in to the accuracy of the classifier with different features, and try to identify misclassified documents -why do you think they were misclassified?**

### Create method for easier comparisons of changes in variables

In [22]:
def createLangClassifier(ngramRange, maxFeatures, cValue, analyzerParam):
    lang_vectorizer = TfidfVectorizer(max_features = maxFeatures, binary=True, ngram_range = ngramRange, analyzer = analyzerParam)
    lang_feature_matrix_train = lang_vectorizer.fit_transform(lang_text_train)
    lang_feature_matrix_dev = lang_vectorizer.transform(lang_text_dev)
    lang_feature_matrix_test = lang_vectorizer.transform(lang_text_test)

    lang_classifier = sklearn.svm.LinearSVC(C = cValue, verbose = 0)
    lang_classifier.fit(lang_feature_matrix_train, lang_label_train)

    print("analyzer={0}, C={1}, n-gram={2}".format(analyzerParam, cValue, ngramRange))
    print("DEV", lang_classifier.score(lang_feature_matrix_dev, lang_label_dev))
    print("TEST", lang_classifier.score(lang_feature_matrix_test, lang_label_test))
    print("TRAIN", lang_classifier.score(lang_feature_matrix_train, lang_label_train))
    print()
    
    return lang_vectorizer, lang_classifier

### Analyzer = "word"

In [23]:
createLangClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "word"
)
createLangClassifier(
    ngramRange = (1,2), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "word"
)
createLangClassifier(
    ngramRange = (1,3), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "word"
)
createLangClassifier(
    ngramRange = (2,2), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "word"
)
createLangClassifier(
    ngramRange = (2,3), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "word"
)
createLangClassifier(
    ngramRange = (3,3), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "word"
)
print()

analyzer=word, C=0.05, n-gram=(1, 1)
DEV 0.934
TEST 0.9392
TRAIN 0.9878

analyzer=word, C=0.05, n-gram=(1, 2)
DEV 0.9366
TEST 0.9406
TRAIN 0.9968

analyzer=word, C=0.05, n-gram=(1, 3)
DEV 0.937
TEST 0.9392
TRAIN 0.993

analyzer=word, C=0.05, n-gram=(2, 2)
DEV 0.759
TEST 0.7616
TRAIN 0.9842

analyzer=word, C=0.05, n-gram=(2, 3)
DEV 0.7484
TEST 0.7522
TRAIN 0.9796

analyzer=word, C=0.05, n-gram=(3, 3)
DEV 0.398
TEST 0.4048
TRAIN 0.9734




### Analyzer = "char"

In [24]:
createLangClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "char"
)
createLangClassifier(
    ngramRange = (1,2), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "char"
)
createLangClassifier(
    ngramRange = (1,3), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "char"
)
createLangClassifier(
    ngramRange = (2,2), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "char"
)
createLangClassifier(
    ngramRange = (2,3), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "char"
)
createLangClassifier(
    ngramRange = (3,3), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "char"
)
print()

analyzer=char, C=0.05, n-gram=(1, 1)
DEV 0.8896
TEST 0.8864
TRAIN 0.8978

analyzer=char, C=0.05, n-gram=(1, 2)
DEV 0.9736
TEST 0.9762
TRAIN 0.9832

analyzer=char, C=0.05, n-gram=(1, 3)
DEV 0.979
TEST 0.9826
TRAIN 0.9922

analyzer=char, C=0.05, n-gram=(2, 2)
DEV 0.972
TEST 0.9756
TRAIN 0.9822

analyzer=char, C=0.05, n-gram=(2, 3)
DEV 0.979
TEST 0.9826
TRAIN 0.9928

analyzer=char, C=0.05, n-gram=(3, 3)
DEV 0.9782
TEST 0.9832
TRAIN 0.994




### Analysis of classification 

When increasing the word feature count it should not have an big effect on the result. To me it is very understandable that the uniqueness of the 2 word or longer features would not have an effect on the classification. I still am curious why when using the word analyzer, that the classification would fail. When comparing Finnish to Estonian, the languages might look very similar, but there are major differences. My bet is that when the lenght of a document is short, the classifier might find it hard to find the correct language because the same words might appear in other languages.

When using characters as features the results are quite different. The singel character feature works quite well because of languages having their own letters. 
When the feature lenght is increased, the accuracy is immensely increased. Every language has its own gramar rules. This will make some letter frequencies more common. 

To get the best accuracy, one could combine the score from words and from the characters and take the average value of that. 

### Find misclassified documents

I want to find out why the classifier fails **when using words as features**. Lets look at the confusion matrix and some examples

In [25]:
vect, classifer  = createLangClassifier(
    ngramRange = (1,1), 
    maxFeatures = 100000,
    cValue = 0.05,
    analyzerParam = "word"
)

analyzer=word, C=0.05, n-gram=(1, 1)
DEV 0.934
TEST 0.9392
TRAIN 0.9878



In [26]:
feature_matrix_test = lang_vectorizer.transform(lang_text_test)
predictions_test=classifer.predict(feature_matrix_test)
print("confusion matrix")
print('en', 'es', 'et', 'fi', 'pt')
print(sklearn.metrics.confusion_matrix(lang_label_test,predictions_test, labels=['en', 'es', 'et', 'fi', 'pt']))

confusion matrix
en es et fi pt
[[906   3   4  78   9]
 [  1 981   1   8   9]
 [  2   3 911  84   0]
 [  3   3  34 959   1]
 [  2   6   1  52 939]]


The confusion matrix goes from EN -> ES -> ET -> FI -> PT.

Most of the missclassifications seem to be around Finnish. The model might have been overfitted for finnish when comparing english text. Meanwhile the confusion in classification of Estonian and Finnish texts are understandable. Gramatically they are very similar. Although, the classifier seems to classify more on the Finnish side than estonian. Even some Portugese text have been classified as Finnish. I strongly believe the documents to contain words similar to Finnish language and be short texts.

### Lets look at the missclassified documents

Lets create a class that contains the info needed for our predictions and save them to a list. After that we can look more closely what might have gone wrong.

In [30]:
class LangPred:
    
    def __init__(self, lang, pred, text):
        self.lang = lang
        self.pred = pred
        self.text = text
    
    def showPred(self):
        print("Lang= {0}".format(self.lang)) 
        print("Prediction= {0}".format(self.pred)) 
        print("Text= {0}".format(self.text))
        print()

In [31]:
missclassifed = []

incre = 0 
for text in lang_text_test:
    data = vect.transform([text])
    prediction = classifer.predict(data)
    
    if prediction != lang_label_test[incre]:
        missclassifed.append(LangPred(lang_label_test[incre], prediction, text))
        
    incre += 1

### Lets analyse some misscalssifed documents

In [32]:
missclassifed[0].showPred()
missclassifed[5].showPred()
missclassifed[10].showPred()
missclassifed[15].showPred()
missclassifed[20].showPred()
missclassifed[21].showPred()

Lang= en
Prediction= ['fi']
Text= 09/16/99 09:48 PM

Lang= et
Prediction= ['en']
Text= " More"!

Lang= pt
Prediction= ['fi']
Text= Harmonização« inoportuna»

Lang= en
Prediction= ['fi']
Text= Fascinating.

Lang= pt
Prediction= ['fi']
Text= E o Brasil?

Lang= et
Prediction= ['fi']
Text= “ Jansu on mõnus ja heatahtlik mutrike, huvitav nähtus tantsumaailmas.”



When inspecting a few of the missclassifed documents, we can se that my theory is quite on point. The texts are mostly short and can easily be missclassfied. The first one contains only numbers and should probably be parsed out of the data set. 

## 5. BONUS:  Pick one language that interests you, and one treebank for that language, and try to builda POS tagger for this language.

**BONUS On the address universaldependencies.org, you will find datasets for a bunch of languages. These come in an easy-to-parse, well-documented format. Pick one language that interests you, and one treebank for that language, and try to builda POS tagger for this language. You can use the 4th column “UPOS” [https://universaldependencies.org/format.html](https://universaldependencies.org/format.html) Report on your findings. If you have extra time, try to experiment with various features and see if you can make your accuracy go up. You can check here [https://universaldependencies.org/conll18/results-upos.html](https://universaldependencies.org/conll18/results-upos.html) what the state of the art roughly is for your selected language and treebank. Did you come close?** 

Lets start by using the premade POS labeling base from the course and modify it to our needs.

In [37]:
# this is how you read a file of this kind
# one item per line, empty lines between sequences

from collections import namedtuple

#Same as tuple but the fields are named for convenience
#this says we have four fields
OneWord=namedtuple("OneWord",["id","word","lemma","upos"])

def read_conll2003(f_name):
    """Yield complete sentences"""
    current_sentence=[] #This will be a list of (word), which we accumulate for each sentence
    with open(f_name) as f:
        for line in f:
            line=line.strip() #drop whitespace
            if line.startswith("#"): #let's not worry about these for the time being
                continue
            if not line: #sentence break
                if current_sentence: #if we gathered a sentence, we should yield it, because a new starts
                    yield current_sentence #much like return, but continues past this line once the element has been consumed
                    current_sentence=[] #...and start a new one
                continue
            #if we made it here, we are on a normal line
            columns=line.split() #an actual word line
            columns=columns[:4]
            assert len(columns)==4 #we should have four columns, looking at the data
            current_sentence.append(OneWord(*columns))

#Now just read the data in
sentences_train=list(read_conll2003("danish/da_ddt-ud-train.conllu"))
sentences_dev=list(read_conll2003("danish/da_ddt-ud-dev.conllu"))

print("First three sentences")
for sent in sentences_dev[:3]:
    print(sent)
    print()

First three sentences
[OneWord(id='1', word='Hvor', lemma='hvor', upos='ADV'), OneWord(id='2', word='kommer', lemma='komme', upos='VERB'), OneWord(id='3', word='julemanden', lemma='julemand', upos='NOUN'), OneWord(id='4', word='fra', lemma='fra', upos='ADP'), OneWord(id='5', word='?', lemma='?', upos='PUNCT')]

[OneWord(id='1', word='Flertallet', lemma='flertal', upos='NOUN'), OneWord(id='2', word='lever', lemma='leve', upos='VERB'), OneWord(id='3', word='stadig', lemma='stadig', upos='ADV'), OneWord(id='4', word='under', lemma='under', upos='ADP'), OneWord(id='5', word='plastikstykker', lemma='plastikstykke', upos='NOUN'), OneWord(id='6', word='eller', lemma='eller', upos='CCONJ'), OneWord(id='7', word='tæpper', lemma='tæppe', upos='NOUN'), OneWord(id='8', word=',', lemma=',', upos='PUNCT'), OneWord(id='9', word='som', lemma='som', upos='PRON'), OneWord(id='10', word='de', lemma='de', upos='PRON'), OneWord(id='11', word='har', lemma='have', upos='AUX'), OneWord(id='12', word='spændt',

In [38]:
def generate_sentence_features(sent):
    #Generate the features for every word
    #The result should be a list of same length as the sentence
    #Each item is a dictionary of {"feature name"->feature value} mappings, holding all features of the word at that position
    
    sent_features=[] #this will be the result
    for one_word in sent:
        #We do nothing with the rest of the data
        #it just happens to be around
        word_features={}
        word_features["word_"+one_word.word]=1 #the word itself is a feature
        sent_features.append(word_features)
    return sent_features

print(generate_sentence_features(sentences_dev[0])  )

[{'word_Hvor': 1}, {'word_kommer': 1}, {'word_julemanden': 1}, {'word_fra': 1}, {'word_?': 1}]


In [39]:
def prep_data(sentences):
    all_labels=[] #here we gather labels for all words in all sentences
    all_features=[] #here we gather features for all words in all sentences
    for sentence in sentences:
        sent_features=generate_sentence_features(sentence)
        assert len(sent_features)==len(sentence)
        #Now we can get, for every position its label and its features
        for one_word,features in zip(sentence,sent_features):
            all_labels.append(one_word.upos) #label
            all_features.append(features)         #and features to go with it
    return all_labels, all_features

train_labels,train_features=prep_data(sentences_train)
dev_labels,dev_features=prep_data(sentences_dev)

In [40]:
from sklearn.feature_extraction import DictVectorizer
vectorizer=DictVectorizer()
vectorizer.fit(train_features)
print("Vectorizer vocab size:",len(vectorizer.vocabulary_))

feature_vectors_train=vectorizer.transform(train_features)
feature_vectors_dev=vectorizer.transform(dev_features)

print("Train shape",feature_vectors_train.shape)
print("Dev shape",feature_vectors_dev.shape)

Vectorizer vocab size: 16330
Train shape (80378, 16330)
Dev shape (10332, 16330)


In [47]:
classifier=sklearn.svm.LinearSVC(C=0.05,verbose=1)
classifier.fit(feature_vectors_train, train_labels)

[LibLinear]

LinearSVC(C=0.05, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=1)

In [48]:
classifier.score(feature_vectors_dev,dev_labels)

0.801296941540844

Now this 80 score is pretty good for such a basic classifier. Lets try to use the features of the words to the right and left to determine the POS label more accurately. 

In [50]:
def generate_sentence_features(sent):
    #Generate the features for every word
    #The result should be a list of same length as the sentence
    #Each item is a dictionary of {"feature name"->feature value} mappings, holding all features of the word at that position
    
    sent_features=[] #this will be the result
    for word_idx, one_word in enumerate(sent):
        word_features={}
        word_features["word_"+one_word.word]=1 #the word itself is a feature
        if word_idx!=0:
            word_features["left_word_"+sent[word_idx-1].word]=1
        if word_idx!=len(sent)-1:
            word_features["right_word_"+sent[word_idx+1].word]=1
        sent_features.append(word_features)
    return sent_features

train_labels,train_features=prep_data(sentences_train)
dev_labels,dev_features=prep_data(sentences_dev)
vectorizer=DictVectorizer()
vectorizer.fit(train_features)
feature_vectors_train=vectorizer.transform(train_features)
feature_vectors_dev=vectorizer.transform(dev_features)

print("Train shape",feature_vectors_train.shape)
print("Dev shape",feature_vectors_dev.shape)

classifier=sklearn.svm.LinearSVC(C=1,verbose=1)
classifier.fit(feature_vectors_train, train_labels)
classifier.score(feature_vectors_dev,dev_labels)

Train shape (80378, 47795)
Dev shape (10332, 47795)
[LibLinear]

0.9124080526519551

A 91 score is pretty impressive for just adding that feature generator. For just adding the left and right word features increased the POS taggers score by 10 is amazing. Lets compare this to some of the state of the art POS tagger results are.

When comparing the POS taggers from [https://universaldependencies.org/conll18/results-upos.html](https://universaldependencies.org/conll18/results-upos.html) we can easily se that the average of the top taggers for danish is about 97. That is a huge difference in POS tagging compared to the basic one above.

If I had more time, I would have probably tried to make a feature changer based on gramar rules. For example that a certain tag can never follow another certain tag, because that would not be gramatically correct. Applying other of these rules would probably fix some of the missclassified features. 